In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hydrant.topology.geom as gm
import subprocess
import os
from   shapely.geometry import Point
import yaml
import shutil
import warnings
import networkx as nx
import re
import copy


warnings.filterwarnings("ignore")

from riverlakenetwork import BurnLakes


In [2]:
# Load general configurations from a YAML file
yaml_file = os.path.abspath('../code/HydroLakes_MERITBasins/HydroLakes_MERITBasins_Config.yaml')
with open(yaml_file, 'r') as f:
    GeneralConfig = yaml.safe_load(f)

# get the path to the merit-basins files
path_out             = GeneralConfig['DomainFolder']
TempFolder           = GeneralConfig['TempFolder']
pfafs                = GeneralConfig['pfafs']
lake_file            = GeneralConfig['LakeFile']
IncludeLakes         = GeneralConfig['IncludeLakes']
riv_path             = GeneralConfig['MERITBasinsPaths']['RivPath']
cat_path             = GeneralConfig['MERITBasinsPaths']['CatPath']
cst_path             = GeneralConfig['MERITBasinsPaths']['CstPath']
riv_file_template    = GeneralConfig['MERITBasinsPaths']['RivFileTemplate']
cat_file_template    = GeneralConfig['MERITBasinsPaths']['CatFileTemplate']
cst_file_template    = GeneralConfig['MERITBasinsPaths']['CstFileTemplate']

In [3]:
if IncludeLakes:
    #load hydrolakeDataset
    lakeO = gpd.read_file(lake_file) # read the hydrolake dataset
    lakeO = lakeO.loc[lakeO["Hylak_id"] < 200].reset_index(drop=True)
    #lakeO = lakeO.sample(frac=0.10, random_state=42)
    #lakeO = gpd.read_file('lake.gpkg')
    #lakeO = FixHydroLakesForMerit(lakeO) # correct for hydrolakes for merit
    lakeO['x'] = lakeO.centroid.x # add centroid lon
    lakeO['y'] = lakeO.centroid.y # add centroid lat

# loop over the config files:
for pfaf in ['72']:

    # get the list of pfaf for the subdomain
    subdomain = 'pfaf'+pfaf
    
    # create the folder and domain to include
    path_out_temp = TempFolder+'pfaffff'+pfaf+'/'+subdomain+'/'
    if os.path.isdir(path_out_temp):
        try:
            shutil.rmtree(path_out_temp)  # Remove the entire directory and its contents
        except OSError as e:
            print(f"Error: {path_out_temp} - {e}")
    if not os.path.isdir(path_out_temp):
        os.makedirs(path_out_temp)

    # read in the files
    riv, cat = gm.merit_read_file ([pfaf],
                                   riv_path,
                                   riv_file_template,
                                   cat_path,
                                   cat_file_template,
                                   path_cst = cst_path,
                                   cst_file_template = cst_file_template)
    
    # prepare the ntopo for merit
    riv, cat = gm.prepare_ntopo(riv=riv,\
                                riv_cols={'id':'COMID', 'next_id':'NextDownID'},\
                                cat=cat,\
                                cat_cols={'id':'COMID', 'hillslope':'hillslope', 'area' :'unitarea'},\
                                network = 'merit')

In [4]:
# use DataLoader to
# Example config dictionary
config = {
    "riv": riv,
    "riv_dict": {
        "COMID": {"col":"COMID"},
        "NextDownCOMID": {"col":"NextDownID"},
        "length": {"col":"lengthkm"},
        "uparea": {"col":"uparea","unit":"km2"}
    },
    "cat": cat,
    "cat_dict": {
        "COMID": {"col":"COMID"},
        "unitarea": {"col":"unitarea","unit":"km2"},
    },
    "lake": lakeO,
    "lake_dict": {
        "LakeCOMID": {"col":"Hylak_id"},
        "unitarea": {"col":"Lake_area","unit":"km2"}
    }
}

bl = BurnLakes(config)


Input loader started at : 2025-12-16 21:43:05
riv: Loaded
riv_dict: {'COMID': {'col': 'COMID'}, 'NextDownCOMID': {'col': 'NextDownID'}, 'length': {'col': 'lengthkm'}, 'uparea': {'col': 'uparea', 'unit': 'km2'}}
cat: Loaded
cat_dict: {'COMID': {'col': 'COMID'}, 'unitarea': {'col': 'unitarea', 'unit': 'km2'}}
lake: Loaded
lake_dict: {'LakeCOMID': {'col': 'Hylak_id'}, 'unitarea': {'col': 'Lake_area', 'unit': 'km2'}}
Input loader finished at: 2025-12-16 21:43:05
Input loader took      : 0:00:00.021465
Input checker started at : 2025-12-16 21:43:05
Subbasin and lake area units are consistent: km2
riv CRS: EPSG:4326
cat CRS: EPSG:4326
lake CRS: EPSG:4326
Input checker finished at: 2025-12-16 21:43:05
Input checker took      : 0:00:00.081997
Resolving lakes started at : 2025-12-16 21:43:05
Index(['LakeCOMID', 'unitarea_left', 'geometry', 'x', 'y', 'index_right',
       'COMID', 'unitarea_right'],
      dtype='object')
==== Number of lakes after subsetting: 20 ====
==== Number of lakes after r

In [5]:
bl.cat.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/cat1.gpkg")
bl.riv.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/riv1.gpkg")
bl.lake.to_file("/Users/shg096/Desktop/LakeRiverOut/Tempdomain/pfaffff71/lake1.gpkg")

In [6]:
print(bl.riv[bl.riv["COMID"] == 72045904])

          COMID  NextDownCOMID        length        uparea  \
45903  72045904     72080899.0  6.467968e-15  178042.32421   

                                                geometry  LakeCOMID  unitarea  \
45903  LINESTRING (-84.66083 45.8025, -84.66083 45.80...        NaN  9.983632   

       exorheic  endorheic  islake  ...  up1318  up1319  up1320  up1321  \
45903       NaN        NaN     NaN  ...       0       0       0       0   

       up1322  up1323  up1324  up1325  up1326  up1327  
45903       0       0       0       0       0       0  

[1 rows x 1341 columns]


In [7]:
lakeO

,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,Shore_dev,...,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,geometry,x,y
0,1,Caspian Sea,Russia,Europe,SWBD,1,0,377001.91,15829.37,7.27,...,8110.642,107883.0,-29,-1.00,1404108.0,47.717708,45.591934,"POLYGON ((49.96181 37.43847, 49.96457 37.44022...",50.735479,41.713338
1,2,Great Bear,Canada,North America,CanVec,1,0,30450.64,5331.72,8.62,...,535.187,47577.7,145,-1.00,147665.4,-123.505546,65.138384,"POLYGON ((-119.78782 67.03574, -119.78637 67.0...",-120.967234,65.999359
2,3,Great Slave,Canada,North America,CanVec,1,0,26734.29,9814.16,16.93,...,4350.692,4203.2,148,-1.00,995312.3,-117.617115,61.311727,"POLYGON ((-109.93976 62.95851, -109.93831 62.9...",-113.742931,61.784589
3,4,Winnipeg,Canada,North America,CanVec,3,709,23923.04,4018.32,7.33,...,2244.727,1464.3,215,-1.00,919611.5,-97.863542,53.696359,"POLYGON ((-98.80636 53.88021, -98.80578 53.880...",-97.806211,52.532676
4,5,Superior,United States of America,North America,SWBD,1,0,81843.92,5248.36,5.18,...,2869.953,48410.3,179,-1.00,209219.5,-84.460547,46.468593,"POLYGON ((-90.7225 46.6574, -90.72458 46.65783...",-87.756530,47.526524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,195,None,Canada,North America,CanVec,1,0,126.24,142.30,3.57,...,12.512,15813.0,26,17.76,1393.6,-69.682132,70.485899,"POLYGON ((-69.60114 70.52273, -69.59384 70.521...",-70.122029,70.425186
195,196,None,Canada,North America,CanVec,1,0,327.01,387.54,6.05,...,12.773,13757.3,538,5.60,1573.7,-73.436694,70.467416,"POLYGON ((-73.06774 70.80526, -73.06053 70.804...",-73.633414,70.543844
196,197,Neergaure,Canada,North America,CanVec,1,0,166.37,353.51,7.73,...,7.165,1563.6,14,1.07,920.4,-79.684615,70.266809,"POLYGON ((-79.95993 70.31561, -79.9592 70.3155...",-79.949296,70.229900
197,198,None,Canada,North America,CanVec,1,0,103.06,129.80,3.61,...,0.891,26392.2,97,3.47,263.9,-106.492381,70.256250,"POLYGON ((-106.5563 70.36015, -106.55519 70.36...",-106.414979,70.288995


In [8]:
len([72080897, 72080899])

2